In [275]:
import config
import pandas as pd
import json
import pickle
import csv

In [113]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [252]:
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

# BreweryDB

In [13]:
def get_brewery_info(url, api_key, page, page_limit):
    """function to pull the information from the Brewery DB API"""
    paged_data = []
    while page <= page_limit:
        url_params = {'key': api_key, 'format':'json', 'p': page, 'withIngredients': 'Y'}
        response = requests.get(url, params=url_params)
        paged_data.append(response.json())
        page += 1
    return paged_data

In [14]:
api_key = config.key
# url_params = {'key': api_key, 'format':'json', 'p': page, 'withBreweries': 'Y', 'withIngredients': 'Y'}
url = "https://sandbox-api.brewerydb.com/v2/beers/"

In [26]:
data = get_brewery_info(url, api_key, 1, 50)

In [13]:
data[22].keys()

dict_keys(['currentPage', 'numberOfPages', 'totalResults', 'data', 'status'])

In [27]:
data[1]['data'][3]['name']

'American Summer Hoppy Wit'

In [47]:
#for now I just want the name
def get_names(data):
    list_names = []
    for page in data:
        for item in page['data']:
            print(item['name'])
#             list_names.append(item['name'])
    return list_names

In [49]:
list_names = get_names(data)

KeyError: 'data'

In [50]:
list_names

["'Murican Pilsner",
 '11.5° PLATO',
 '12th Of Never',
 '15th Anniversary Ale',
 '16 So Fine Red Wheat Wine',
 '1794 The Fergal Project',
 '17th Saison',
 '18th Anniversary Belgian Tripel',
 '19 - Golden Belgian Style Ale',
 '1904 American Red Lager',
 '2 x 4',
 '200th Anniversary Export Stout',
 '2017 Beer Camp',
 '20th Anniversary Imperial Hash IPA on Brett',
 '20th Street Ale Citra',
 '20th Street Ale Crystal',
 '20th Street Ale Magnum',
 '21st Anniversary',
 '25th Anniversary',
 '3 Weight',
 "30th Anniversary - Charlie, Fred & Ken's Bock",
 "30th Anniversary - Fritz and Ken's Ale",
 '30th Anniversary - Grand Cru',
 "30th Anniversary - Jack & Ken's Ale",
 "35th Anniversary  - Brewer's Reserve",
 '420 Extra Pale Ale',
 '420 Fest',
 '420 Strain G13 IPA',
 '471 Double IPA - Hull Melon',
 '471 ESB - Extra Special Bitter',
 '471 IPA Barrel Series: Citra',
 '471 IPA Barrel Series: Eureka!',
 '471 IPA. Aggressive Hoppiness',
 '471 Pilsner',
 '471 Small Batch IPA',
 "5 C's IPA",
 '7 Birds',

In [17]:
def parse(data_lst):
    # elements of the style keys list to look for when we parse through style 
    style_lst = ["id", "name", "shortName",'abvMax','srmMin', 'srmMax', "ibuMax", "ibuMin",
                 'ogMin','fgMin','fgMax' ]
    api_dict_list = []
    # read_through each in the data_list
    for index in range(0,23):
        print(index)
        for api_call in data_lst[index]["data"]:
            api_dict = {}
            api_dict["name"] = api_call["name"]
            if 'abv' in api_call:
                api_dict["abv"] = float(api_call["abv"])
            if 'ibu' in api_call:
                api_dict['ibu'] = float(api_call["ibu"])
            if api_call['isOrganic'] == 'Y':
                api_dict["is_organic"] = 1
            else:
                api_dict['is_organic'] = 0
            api_dict["created"] = api_call["createDate"]
            if "style" in api_call:
                for style in style_lst:
                    if style in api_call["style"]:
                        api_dict[f"style_{style}"] = api_call["style"][style]
                api_dict["category_name"] = api_call["style"]["category"]["name"]
                api_dict["category_id"] = api_call["style"]["category"]["id"]
            brewery = api_call["breweries"][0]
            api_dict[f"brewery_name"] =  brewery["name"]
            api_dict[f"brewery_id"] = brewery["id"]
            if "established" in api_call["breweries"]:
                api_dict[f"established"] = brewery["established"]
            api_dict['location'] = brewery["locations"][0]['region']
#             api_dict["brewery_locations_map"] = []
#             api_dict["brewery_locations"] = []
#             api_dict["brewery_locality"] = ""
#             count = 1
#             for idx in range(len(brewery["locations"])):
#                 brew_location = brewery["locations"][0]
#                 api_dict["brewery_locations"].append(brew_location["name"])
#                 if "latitude" in brew_location and "longitude" in brew_location:
#                     api_dict["brewery_locations_map"].append([float(brew_location["latitude"]),
#                                                              float(brew_location["longitude"])])
#                 if "locality" in brew_location:
#                     api_dict["brewery_locality"]+= f'{count}. {brew_location["locality"]}' + " "
#                     count += 1
#             api_dict["number_of_locations"] = len(api_dict["brewery_locations"])
            api_dict_list.append(api_dict)
    return api_dict_list

# Beer Advocate

In [63]:
driver = webdriver.Chrome()

In [64]:
driver.get('https://www.beeradvocate.com/community/login/')

In [67]:
driver.find_element_by_xpath('//*[@id="navigation"]/div/div/div/div/nav/div/ul[1]/li[5]/a[1]').click()

In [69]:
comment = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]/div[1]').text

In [70]:
comment

'"Garden Party is a blend of sour blond and wheat ales aged in white wine barrels for up to two years with Willamette Valley-grown chardonnay grapes followed by a late infusion of fresh Meyer lemon zest and dried elderflowers. The perfect beverage for any outdoor soirée, Garden Party delivers the sparkle of champagne, the floral delicacy of elderflowers and the brilliance of sweet lemon zest—bringing refinement and refreshment to your party, wherever it may be."\n\n5 oz pour on tap. The first sip yields heavy funk and a surprising amount of sweetness - not a great combination. However, the sweetness mellows out and gives way to oak, lemon, vanilla, white grape juice, and a floral component that must be the elderflowers. A unique beer for sure and decently complex. Not my favorite, but better than it seemed at first sip.'

In [71]:
atts = driver.find_element_by_xpath('//*[@id="rating_fullview_content_2"]/span[5]').text

In [72]:
atts

'look: 3.75 | smell: 4 | taste: 4.25 | feel: 4 | overall: 4'

## straight to beer url:

In [77]:
name = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div/div[1]/h1').text

In [81]:
name.split('\n')[1]

'Sierra Nevada Brewing Co.'

In [75]:
img = driver.find_element_by_xpath('//*[@id="main_pic_mobile"]/div/img')
src = img.get_attribute('src')

In [76]:
src

'https://cdn.beeradvocate.com/im/beers/276.jpg'

In [ ]:
beer = {}
beer['name'] = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div/div[1]/h1').text


In [99]:
listy = driver.find_elements_by_xpath('//*[@id="rating_fullview_content_2"]')

In [100]:
for item in listy:
    print(item.text)

4/5  rDev -1.2%
look: 4 | smell: 4 | taste: 4 | feel: 4 | overall: 4

11/9/19 - Single from Elkton; did a comparison with Monument City Pagoda (pale ale). Both are very nice. Monument is a bit sharper compared to the milder SNPA, but very nice as well.

 182 characters

DuaneL, Nov 09, 2019  Report
4.71/5  rDev +16.3%
look: 4.5 | smell: 4.75 | taste: 4.75 | feel: 4.5 | overall: 4.75

An exceptional pale ale by any standard. Golden amber in colour, pours well with nice head. Really enjoyable citrus aroma and flavour that lingers but isn't bitter at the end like others I've tried. Smooth and drinkable. Overall one of my favourites.

 254 characters

MMoore1971, Nov 05, 2019  Report
3.83/5  rDev -5.4%
look: 3.75 | smell: 3.75 | taste: 4 | feel: 3.5 | overall: 3.75

Pours golden with a 1/2 inch white head. Moderate lacing as you go.

Aroma is grapefruity, piney, resiny, grassy.

Tastes of strong pine and grapefruit. Good bitterness.

Medium bodied, dry mouthfeel. Light carbonation.

While 

4.24/5  rDev +4.7%
look: 4.25 | smell: 4 | taste: 4.25 | feel: 4.25 | overall: 4.5

12oz bottle - dated 03/05/19 - poured into a Spiegelau-style IPA glass at 45F

Pours a bright copper hue, nearly crystal clear, with about two fingers of very off white head, which possesses impressive retention, never fully dissipating to less than a full cap, and leaving a perfect ring of clingy lace behind. Fine ascending streams of effervescence are visible.

Nose: A hefty base of caramel malt is apparent on the nose. Slightly sweet, slightly toasty, crisp, and warm. Among the malt, a generous dose of Cascade hops waft up, with aromas of grapefruit, lemon, pine resin, pear, and perhaps even a note of sweet ripe mango. There is no earthiness, but rather a green element to the nose that evokes images of a forest. Lots of floral notes as well, including lilac, hyacinth, and Easter lilly.

Palate: Like the nose, the first sip unfolds with caramel malt. Semi-sweet, slightly toasty, and crisp, with notes 

In [105]:
listy = driver.find_elements_by_class_name('user-comment')

In [107]:
listy_text = []
for item in listy:
    listy_text.append(item.text)

In [108]:
listy_text

['4/5  rDev -1.2%\nlook: 4 | smell: 4 | taste: 4 | feel: 4 | overall: 4\n\n11/9/19 - Single from Elkton; did a comparison with Monument City Pagoda (pale ale). Both are very nice. Monument is a bit sharper compared to the milder SNPA, but very nice as well.\n\n 182 characters\n\nDuaneL, Nov 09, 2019  Report',
 "4.71/5  rDev +16.3%\nlook: 4.5 | smell: 4.75 | taste: 4.75 | feel: 4.5 | overall: 4.75\n\nAn exceptional pale ale by any standard. Golden amber in colour, pours well with nice head. Really enjoyable citrus aroma and flavour that lingers but isn't bitter at the end like others I've tried. Smooth and drinkable. Overall one of my favourites.\n\n 254 characters\n\nMMoore1971, Nov 05, 2019  Report",
 "3.83/5  rDev -5.4%\nlook: 3.75 | smell: 3.75 | taste: 4 | feel: 3.5 | overall: 3.75\n\nPours golden with a 1/2 inch white head. Moderate lacing as you go.\n\nAroma is grapefruity, piney, resiny, grassy.\n\nTastes of strong pine and grapefruit. Good bitterness.\n\nMedium bodied, dry mout

In [52]:
def geturl(beer):
    driver.get('https://www.beeradvocate.com/beer/')
    time.sleep(.5)
    search_butt = driver.find_element_by_xpath('//*[@id="QuickSearchPlaceholder"]')
    search_butt.click()
    search_beer_name=driver.find_element_by_xpath('//*[@id="QuickSearchQuery"]')
    search_beer_name.send_keys(beer)
    click_search=driver.find_element_by_xpath('//*[@id="QuickSearch"]/form/div[2]/dl/dd/input').click()
    url=driver.current_url
    if url.split('/')[3]=='beer':
        return driver.current_url
    else: 
        try:
            driver.find_element_by_xpath('//*[@id="ba-content"]/div[1]/span/li').text.split('.')[0]=='No results'
            return None
        except:
            
            driver.find_element_by_xpath('//*[@id="ba-content"]/div[2]/div/a[1]/span').click()
            url=driver.current_url
            return url

In [5]:
geturl('sierra nevada pale ale')

'https://www.beeradvocate.com/beer/profile/140/276/'

In [9]:
def get_beer_info(list_urls):
    list_info = []
    for url in list_urls:
        print(url)
        page = requests.get(url)
        soup = BS(page.content, 'html.parser')
        data = soup.findAll(class_ = 'beerstats')
        rating = soup.find(class_ = 'ba-ravg Tooltip').get_text()
        if not soup.find(class_ = 'ba-score Tooltip'):
            score = None
        else:
            score = soup.find(class_ = 'ba-score Tooltip').get_text()
        no_reviews = soup.find(class_ = 'ba-reviews Tooltip').get_text()
        no_ratings = soup.find(class_ = 'ba-ratings Tooltip').get_text()
        if len(soup.findAll(title="Percentage of alcohol by volume.")) == 1:
            abv = None
        else:
            abv = soup.findAll(title="Percentage of alcohol by volume.")[1].get_text()
        avail = soup.findAll(title="The beer's availability.")[1].get_text()
        style = soup.find(title="Learn more about this style.").get_text()
        state = [d.get_text().split(',')[0] for d in data if d.get_text().split(',')[0] in states]
        if len(state) == 0:
            place = 'World'
        else:
            place = state[0]
        beer_info = (rating, score, no_reviews, no_ratings, abv, avail, style, place)
        list_info.append(beer_info)
    return list_info

In [11]:
def do_everything(list_beer_names):
    urls = []
    for beer in list_beer_names:
        urls.append(geturl(beer))
    return urls

In [54]:
list_urls = do_everything(list_names[1])

KeyboardInterrupt: 

In [55]:
list_names[1]

'11.5° PLATO'

In [62]:
driver.quit()

# BS to get urls

In [124]:
url = 'https://www.beeradvocate.com/beer/styles/'
page = requests.get(url)
soup = BS(page.content, 'html.parser')

In [126]:
url = 'https://www.beeradvocate.com/beer/styles/'
page = requests.get(url)
data = page.text
soup = BS(data, 'html.parser')

In [128]:
soup

<!DOCTYPE html>

<html class="Public NoJs uix_javascriptNeedsInit LoggedOut Sidebar Responsive pageIsLtr not_hasTabLinks hasSearch is-sidebarOpen hasRightSidebar is-setWidth navStyle_0 pageStyle_0 hasFlexbox" dir="LTR" id="XenForo" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<base href="https://www.beeradvocate.com/community/"/>
<script>
			var _b = document.getElementsByTagName('base')[0], _bH = "https://www.beeradvocate.com/community/";
			if (_b && _b.href != _bH) _b.href = _bH;
		</script>
<title>Beer Styles | BeerAdvocate</title>
<noscript><style>.JsOnly, .jsOnly { display: none !important; }</style></noscript>
<link href="css.php?css=xenforo,form,public&amp;style=6&amp;dir=LTR&amp;d=1574286681" rel="stylesheet"/>
<link href="css.php?css=login_bar,moderator_bar,uix_extendedFooter&amp;style=6&amp;dir=LTR&

In [119]:
tags = soup.find_all('a')

In [120]:
for tag in tags:
    if
    print(tag.get('href'))

#
/
/
/about/
/advertise/
/contact/
https://www.beeradvocate.com/community/
https://www.beeradvocate.com/community/
search/?type=post
find-new/posts
/beer/
/beer/
/user/beers/
/forms/add-beer/
/beer/styles/
/beer/new/
/trading/
/beer/top-rated/
/beer/top-rated/
/beer/top-rated/
/beer/top-rated/us/
/beer/top-styles/
/beer/trending/
/beer/top-new/
/beer/fame/
/beer/popular/
/place/
/place/
/user/places/
/toolbox/add_place/
/place/directory/
/place/nearby/
/society/
/community/threads/frequently-asked-questions-faq.444392/
/community/threads/frequently-asked-questions-faq.444392/
/about/
/contact/
/community/forums/help-resources.32/
/community/forums/beeradvocate-talk.34/
/community/threads/how-to-claim-your-place.553982/
#
lost-password/
#
https://www.beeradvocate.com
https://www.beeradvocate.com
#
/
/
/about/
/advertise/
/contact/
https://www.beeradvocate.com/community/
https://www.beeradvocate.com/community/
search/?type=post
find-new/posts
/beer/
/beer/
/user/beers/
/forms/add-beer/


In [135]:
list_beer_style_urls = ['/beer/styles/32/','/beer/styles/35/','/beer/styles/36/','/beer/styles/33/',
                        '/beer/styles/92/','/beer/styles/73/','/beer/styles/74/','/beer/styles/75/',
                        '/beer/styles/86/','/beer/styles/175/','/beer/styles/119/','/beer/styles/57/',
                        '/beer/styles/18/','/beer/styles/68/','/beer/styles/47/','/beer/styles/147/',
                        '/beer/styles/149/','/beer/styles/29/','/beer/styles/7/','/beer/styles/31/',
                        '/beer/styles/46/','/beer/styles/30/','/beer/styles/6/','/beer/styles/141/',
                        '/beer/styles/114/','/beer/styles/132/','/beer/styles/199/','/beer/styles/140/',
                        '/beer/styles/116/','/beer/styles/174/','/beer/styles/150/','/beer/styles/189/',
                        '/beer/styles/128/','/beer/styles/99/','/beer/styles/97/','/beer/styles/190/',
                        '/beer/styles/54/','/beer/styles/129/','/beer/styles/98/','/beer/styles/66/',
                        '/beer/styles/154/','/beer/styles/76/','/beer/styles/127/','/beer/styles/85/',
                        '/beer/styles/161/','/beer/styles/38/','/beer/styles/164/','/beer/styles/155/',
                        '/beer/styles/39/','/beer/styles/42/','/beer/styles/40/','/beer/styles/20/',
                        '/beer/styles/37/',
'/beer/styles/43/',
'/beer/styles/21/',
'/beer/styles/131/',
'/beer/styles/41/',
'/beer/styles/192/',
'/beer/styles/159/',
'/beer/styles/80/',
'/beer/styles/101/',
'/beer/styles/193/',
'/beer/styles/194/',
'/beer/styles/163/',
'/beer/styles/148/',
'/beer/styles/9/',
'/beer/styles/8/',
'/beer/styles/168/',
'/beer/styles/169/',
'/beer/styles/5/',
'/beer/styles/72/',
'/beer/styles/173/',
'/beer/styles/12/',
'/beer/styles/70/',
'/beer/styles/11/',
'/beer/styles/157/',
'/beer/styles/158/',
'/beer/styles/69/',
'/beer/styles/13/',
'/beer/styles/82/',
'/beer/styles/95/',
'/beer/styles/162/',
'/beer/styles/84/',
'/beer/styles/19/',
'/beer/styles/191/',
'/beer/styles/78/',
'/beer/styles/60/',
'/beer/styles/142/',
'/beer/styles/56/',
'/beer/styles/55/',
'/beer/styles/58/',
'/beer/styles/152/',
'/beer/styles/79/',
'/beer/styles/165/',
'/beer/styles/77/',
'/beer/styles/94/',
'/beer/styles/93/',
'/beer/styles/48/',
'/beer/styles/87/',
'/beer/styles/91/',
'/beer/styles/89/',
'/beer/styles/90/',
'/beer/styles/198/',
'/beer/styles/171/',
'/beer/styles/15/',
'/beer/styles/10/',
'/beer/styles/14/',
'/beer/styles/50/',
'/beer/styles/52/',
'/beer/styles/53/',
'/beer/styles/16/']

In [136]:
len(list_beer_style_urls)

111

In [196]:
list_beer_style_urls[2:10]

['/beer/styles/36/',
 '/beer/styles/33/',
 '/beer/styles/92/',
 '/beer/styles/73/',
 '/beer/styles/74/',
 '/beer/styles/75/',
 '/beer/styles/86/',
 '/beer/styles/175/']

In [168]:
url = 'https://www.beeradvocate.com/beer/styles/32'
page = requests.get(url)
soup = BS(page.content, 'html.parser')

In [171]:
tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]

In [173]:
len(tags)

50

In [174]:
tags

['/beer/profile/35/101/',
 '/beer/profile/143/1352/',
 '/beer/profile/35/14309/',
 '/beer/profile/29/1252/',
 '/beer/profile/35/68000/',
 '/beer/profile/590/2565/',
 '/beer/profile/35/74579/',
 '/beer/profile/125/2272/',
 '/beer/profile/337/1070/',
 '/beer/profile/2137/3835/',
 '/beer/profile/302/41422/',
 '/beer/profile/66/1873/',
 '/beer/profile/184/20299/',
 '/beer/profile/428/14899/',
 '/beer/profile/57/7714/',
 '/beer/profile/417/47795/',
 '/beer/profile/35/225611/',
 '/beer/profile/107/7904/',
 '/beer/profile/99/75739/',
 '/beer/profile/1819/31215/',
 '/beer/profile/144/1000/',
 '/beer/profile/15280/39884/',
 '/beer/profile/455/2250/',
 '/beer/profile/30128/86334/',
 '/beer/profile/220/283116/',
 '/beer/profile/1805/20968/',
 '/beer/profile/628/5119/',
 '/beer/profile/410/144046/',
 '/beer/profile/435/6340/',
 '/beer/profile/262/161796/',
 '/beer/profile/35/289887/',
 '/beer/profile/5502/27167/',
 '/beer/profile/7734/76116/',
 '/beer/profile/514/11985/',
 '/beer/profile/31272/171

In [ ]:
for i in range(0, 1150):
    
'https://www.beeradvocate.com/beer/styles/32/?sort=revsD&start=0'

In [175]:
i = 0
list_links = []
for i in range(0, 1150):
    url = f'https://www.beeradvocate.com/beer/styles/32/?sort=revsD&start={i}'
    if 
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
    list_links.append(tags)
    i += 50
    

In [212]:
len(list_links)

1425

In [187]:
with open('first_links.pkl', 'wb') as f:
    pickle.dump(list_links, f)

In [ ]:
# open it again
with open('first_links.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [209]:
url = f"https://www.beeradvocate.com{list_beer_style_urls[1]}"
page = requests.get(url)
soup = BS(page.content, 'html.parser')
for tag in soup.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
#     print(tag.get('href'))
    if tag.contents[0] == 'last':
        num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
        print(num)

1300
1300


In [210]:
num

1300

In [221]:
new_list_links_2 = []
j = 3
for url_end in list_beer_style_urls[0:2]:
    url = f'https://www.beeradvocate.com{url_end}'
    print(f'j is {j}')
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    for tag in soup.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
        if tag.contents[0] == 'last':
            num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
    some_num = num+50
    i = 0
    for i in range(0, some_num, 50):
        print(f'i is {i}')
        url = f'https://www.beeradvocate.com{url_end}?sort=revsD&start={i}'
        page = requests.get(url)
        soup = BS(page.content, 'html.parser')
        tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
        new_list_links.append(tags)
        i += 50
    with open(f'{j}_list_links.pkl', 'wb') as f:
        pickle.dump(list_links, f)
    j += 1
    time.sleep(1)

j is 1
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
j is 2
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300


In [222]:
len(new_list_links)

51

In [223]:
len(list_beer_style_urls)

111

In [224]:
new_list_links_2 = []
j = 3
for url_end in list_beer_style_urls[2:]:
    url = f'https://www.beeradvocate.com{url_end}'
    print(f'j is {j}')
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    for tag in soup.find_all(href=re.compile("/beer/styles/[0-9]+/\?sort=revsD")):
        if tag.contents[0] == 'last':
            num = int(tag.get('href').strip(f"{url}?sort=revsD&start"))
    some_num = num+50
    i = 0
    for i in range(0, some_num, 50):
        print(f'i is {i}')
        url = f'https://www.beeradvocate.com{url_end}?sort=revsD&start={i}'
        page = requests.get(url)
        soup = BS(page.content, 'html.parser')
        tags = [tag.get('href') for tag in soup.find_all(href=re.compile("/beer/profile/[0-9]+/[0-9]+"))]
        new_list_links.append(tags)
        i += 50
    with open(f'{j}_list_links.pkl', 'wb') as f:
        pickle.dump(list_links, f)
    j += 1
    time.sleep(1)

j is 3
i is 0
i is 50
j is 4
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
j is 5
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
j is 6
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
i is 2400
i is 2450
i is 2500
i is 2550
i is 2600
i is 2650
i is 2700
i is 2750
i is 2800
i is 2850
i is 2900
i is 2950
i is 3000
i is 3050
i is 3100
i is 3150
i is 3200
i is 3250
i is 3300
i is 3350

i is 12200
i is 12250
i is 12300
i is 12350
i is 12400
i is 12450
i is 12500
i is 12550
i is 12600
i is 12650
i is 12700
i is 12750
i is 12800
i is 12850
i is 12900
i is 12950
i is 13000
i is 13050
i is 13100
i is 13150
i is 13200
i is 13250
i is 13300
i is 13350
i is 13400
i is 13450
i is 13500
i is 13550
i is 13600
i is 13650
i is 13700
i is 13750
i is 13800
i is 13850
i is 13900
i is 13950
i is 14000
i is 14050
i is 14100
i is 14150
i is 14200
i is 14250
i is 14300
i is 14350
i is 14400
i is 14450
i is 14500
i is 14550
i is 14600
i is 14650
i is 14700
i is 14750
i is 14800
i is 14850
i is 14900
i is 14950
i is 15000
i is 15050
i is 15100
i is 15150
i is 15200
i is 15250
i is 15300
i is 15350
i is 15400
i is 15450
i is 15500
i is 15550
i is 15600
i is 15650
i is 15700
i is 15750
i is 15800
i is 15850
i is 15900
i is 15950
i is 16000
i is 16050
i is 16100
i is 16150
i is 16200
i is 16250
i is 16300
i is 16350
i is 16400
i is 16450
i is 16500
i is 16550
i is 16600
i is 16650
i is 16700

i is 4400
i is 4450
i is 4500
i is 4550
i is 4600
i is 4650
i is 4700
i is 4750
i is 4800
i is 4850
i is 4900
i is 4950
i is 5000
i is 5050
i is 5100
i is 5150
i is 5200
i is 5250
i is 5300
i is 5350
i is 5400
i is 5450
i is 5500
i is 5550
i is 5600
i is 5650
i is 5700
i is 5750
i is 5800
i is 5850
i is 5900
i is 5950
i is 6000
i is 6050
i is 6100
j is 35
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
i is 2400
i is 2450
i is 2500
i is 2550
i is 2600
i is 2650
i is 2700
i is 2750
i is 2800
i is 2850
i is 2900
i is 2950
i is 3000
i is 3050
i is 3100
i is 3150
i is 3200
i is 3250
i is 

i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
j is 50
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
j is 51
i is 0
i is 50
j is 52
i is 0
i is 50
j is 53
i is 0
i is 50
j is 54
i is 0
i is 50
j is 55
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
j is 56
i is 0
i is 50
j is 57
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
j is 58
i is 0
j is 59
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
i is 2400
i is 245

i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
j is 79
i is 0
j is 80
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is 1650
i is 1700
i is 1750
i is 1800
i is 1850
i is 1900
i is 1950
i is 2000
i is 2050
i is 2100
i is 2150
i is 2200
i is 2250
i is 2300
i is 2350
i is 2400
i is 2450
i is 2500
i is 2550
i is 2600
i is 2650
i is 2700
i is 2750
i is 2800
i is 2850
i is 2900
i is 2950
i is 3000
i is 3050
i is 3100
i is 3150
i is 3200
i is 3250
i is 3300
i is 3350
i is 3400
i is 3450
i is 3500
i is 3550
i is 3600
i is 3650
i is 3700
i is 3750
j is 81
i is 0
j is 82
i is 0
i is 50
i is 100
i is 150
i is

i is 6600
i is 6650
i is 6700
i is 6750
i is 6800
i is 6850
i is 6900
i is 6950
i is 7000
i is 7050
i is 7100
i is 7150
i is 7200
i is 7250
i is 7300
i is 7350
i is 7400
i is 7450
i is 7500
i is 7550
i is 7600
i is 7650
i is 7700
i is 7750
i is 7800
i is 7850
i is 7900
i is 7950
i is 8000
i is 8050
i is 8100
i is 8150
i is 8200
i is 8250
i is 8300
i is 8350
i is 8400
i is 8450
i is 8500
i is 8550
i is 8600
i is 8650
i is 8700
i is 8750
i is 8800
i is 8850
i is 8900
i is 8950
i is 9000
i is 9050
i is 9100
i is 9150
i is 9200
i is 9250
i is 9300
i is 9350
i is 9400
i is 9450
i is 9500
i is 9550
i is 9600
i is 9650
i is 9700
i is 9750
i is 9800
i is 9850
i is 9900
i is 9950
j is 105
i is 0
i is 50
i is 100
i is 150
i is 200
i is 250
i is 300
i is 350
i is 400
i is 450
i is 500
i is 550
i is 600
i is 650
i is 700
i is 750
i is 800
i is 850
i is 900
i is 950
i is 1000
i is 1050
i is 1100
i is 1150
i is 1200
i is 1250
i is 1300
i is 1350
i is 1400
i is 1450
i is 1500
i is 1550
i is 1600
i is

In [230]:
len(new_list_links)

4512

In [231]:
with open(f'beer_links.pkl', 'wb') as f:
    pickle.dump(new_list_links, f)

In [234]:
new_list_links[0][2]

'/beer/profile/35/14309/'

In [240]:
4512*50

225600

In [238]:
final_list_links = []
for list_links in new_list_links:
    final_list_links += list_links

In [239]:
len(final_list_links)

213921

In [241]:
final_list_links[0]

'/beer/profile/35/101/'

In [328]:
csv_columns = ['url', 'avg_score', 'ba_score', 'no_reviews', 'no_ratings', 'abv', 'avail', 'style', 'location', 'brewery', 'review', 'img']
csv_file = 'output.csv'

In [438]:
def get_beer_info(list_urls):
    list_beers = []
    i = 1
    for url_end in list_urls:
        beer = {}
        url = f'https://www.beeradvocate.com{url_end}'
        beer['url'] = url
        print(i)
        page = requests.get(url)
        soup = BS(page.content, 'html.parser')
        
        data = soup.findAll(class_ = 'beerstats')
        # average score
        if not soup.find(class_ = 'ba-ravg Tooltip'):
            beer['avg_score'] = None
        else:    
            beer['avg_score'] = soup.find(class_ = 'ba-ravg Tooltip').get_text()
        # ba score
        if not soup.find(class_ = 'ba-score Tooltip'):
            beer['ba_score'] = None
        else:
            beer['ba_score'] = soup.find(class_ = 'ba-score Tooltip').get_text()
        # number of reviews
        if not soup.find(class_ = 'ba-reviews Tooltip'):
            beer['no_reviews'] = None
        else:
            beer['no_reviews'] = soup.find(class_ = 'ba-reviews Tooltip').get_text()
        # number of ratings
        if not soup.find(class_ = 'ba-ratings Tooltip'):
            beer['no_ratings'] = None
        else:
            beer['no_ratings'] = soup.find(class_ = 'ba-ratings Tooltip').get_text()
        # abv
        if len(soup.findAll(title="Percentage of alcohol by volume.")) == 1:
            beer['abv'] = None
        else:
            beer['abv'] = soup.findAll(title="Percentage of alcohol by volume.")[1].get_text()
        # availability
        if not soup.findAll(title="The beer's availability."):
            beer['avail'] = None
        else:
            beer['avail'] = soup.findAll(title="The beer's availability.")[1].get_text()
        # style
        if not soup.find(title="Learn more about this style."):
            beer['style'] = None
        else:
            beer['style'] = soup.find(title="Learn more about this style.").get_text()
        # convluted way of getting the location
        state = [d.get_text().split(',')[0] for d in data if d.get_text().split(',')[0] in states]
        if len(state) == 0:
            beer['location'] = [d.get_text().split(',') for d in data][16][0]
        else:
            beer['location'] = state[0]
        # getting the brewery name
        if not soup.find(title="View more beers and info from this brewery."):
            beer['brewery'] = None
        else:
            beer['brewery'] = soup.find(title="View more beers and info from this brewery.").get_text()
        # get the reviews!
        listy = soup.find_all(class_ = 'user-comment')
        beer['review'] = []
        for item in listy:
            beer['review'].append(item.get_text())
        # bonus image!
        if not soup.find_all('img', {'src':re.compile('.jpg')}):
            beer['img'] = None
        else:
            image = soup.find_all('img', {'src':re.compile('.jpg')})
            beer['img'] = image[0]['src']
        # NAME of the beer
        titlebar = soup.find(class_='titleBar')
        for span in titlebar('span'):
            span.decompose()

        beer['name'] = titlebar.get_text().strip()
        
        list_beers.append(beer)
    
        with open('output1.json', 'w', encoding='utf-8') as file:
            json.dump(list_beers, file)

        time.sleep(0.5)
        i += 1
    return list_beers

In [416]:
first = get_beer_info(final_list_links[0:2])

https://www.beeradvocate.com/beer/profile/35/101/
https://www.beeradvocate.com/beer/profile/143/1352/


In [417]:
dff = pd.read_json('output.json')

In [418]:
dff

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
0,5.6%,Winter,3.60,82,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",[3.31/5 rDev -8.1%look: 4 | smell: 3.5 | tast...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/
1,4.4%,Year-round,3.19,73,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163",[3.19/5 rDev 0%look: 3.75 | smell: 2.75 | tas...,German Bock,https://www.beeradvocate.com/beer/profile/143/...


In [366]:
dff['review'][1]

["3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.75 | taste: 3.25 | feel: 3.5 | overall: 3.25I first sampled this beer when I visited my Father in Texas earlier this year, and I had a can kicking around in the fridge that I brought back with me. \n\nBocks are a pretty underrated style in my book, partly because so few brewers in Alberta put them out - let alone put out any good ones. While this beer is nothing spectacular at the end of the day, I found myself buying a couple of cases of this from a nearby HEB to drink while I was in Austin. It's such a simple product, but something about it just appeals to me. It has a pleasant mild maltiness, with decidedly a stronger flavour than your run-of-the-mill domestics. No nasty adjunct flavours, and pours an attractive light brown colour. It's a beer that you can - and I did - drink several of in one go. \n\nI can understand why this beer is so popular in Texas. Just a shame they haven't exported it across the border.\xa0879 charactersOlek4374, 

In [305]:
final_list_links[0:1]

['/beer/profile/35/101/']

In [411]:
url = 'https://www.beeradvocate.com/beer/profile/38985/450790/'
print(url)
page = requests.get(url)
soup = BS(page.content, 'html.parser')
data = soup.findAll(class_ = 'beerstats')
# print(data)
# rating = soup.find(class_ = 'ba-ravg Tooltip').get_text()

https://www.beeradvocate.com/beer/profile/38985/450790/


In [412]:
image = soup.find_all('img', {'src':re.compile('.jpg')})
print(image[0]['src'])

https://cdn.beeradvocate.com/im/placeholder-beer.jpg


In [413]:
soup.find(class_ = 'titleBar')

<div class="titleBar">
<h1>Dry &amp; Bitter / Interboro Juicy Gotcha Back<br/><span style="color:#999999; font-size:0.75em;">Dry &amp; Bitter Brewing Company</span></h1>
</div>

In [406]:
brewery = soup.find(title="View more beers and info from this brewery.").get_text()

In [414]:
titlebar = soup.find(class_='titleBar')
for span in titlebar('span'):
    span.decompose()

print(titlebar.get_text().strip())


Dry & Bitter / Interboro Juicy Gotcha Back


In [391]:
soup.find('img', {'src':re.compile('.jpg')})

<img alt="" border="0" src="https://cdn.beeradvocate.com/im/placeholder-beer.jpg" style="width:100%;"/>

In [375]:
for item in soup.find_all(class_ = 'user-comment'):
    print(item.get_text())

3.31/5  rDev -8.1%look: 4 | smell: 3.5 | taste: 3 | feel: 3.25 | overall: 3.5Alright - this is one of those ‘it depends’ beers. Great deep copper, molasses color and pours with a brief head. Nice mouth feel with a good swallow. LET IT WARM UP A BIT is my best advice. Doesn’t do much when super cold, but a nice bread malt aroma...maybe a touch of spice is in there if you give it a chance. Not over the top and a reasonable bock buuuut a touch of hops may set some people off. I like this beer with dinner when I’m out eating warm, winter comfort food. Not a 4-5 range beer to buy to take home but a nice beverage on a cold night out. 566 charactersWilliamGustin, Yesterday at 11:04 PM
2.74/5  rDev -23.9%look: 3.5 | smell: 2.5 | taste: 2.75 | feel: 2.75 | overall: 2.75Very pretty copper color with no head to speak of.
Smell was disappointing. Maybe toast if I use my imagination...maybe.
The first thing I tasted was hops. And if you like hops, that's great. But I don't love them, and Sam Adams 

In [299]:
images = soup.find_all('img', {'src':re.compile('.jpg')})
print(images[0]['src'])
# for image in images:
#     print(image['src']+'\n')

https://cdn.beeradvocate.com/im/beers/101.jpg


In [293]:
images

[<img alt="Guinness Draught" border="0" src="https://cdn.beeradvocate.com/im/beers/754.jpg" style="width:100%;"/>,
 <img alt="Guinness Draught" border="0" src="https://cdn.beeradvocate.com/im/beers/754.jpg" style="width:100%;"/>,
 <img alt="Photo of Lanas" border="0" height="48" src="https://cdn.beeradvocate.com/data/avatars/s/1276/1276607.jpg?1567421012" width="48"/>,
 <img alt="Photo of Handsome_Steve" border="0" height="48" src="https://cdn.beeradvocate.com/data/avatars/s/1210/1210089.jpg?1516479491" width="48"/>,
 <img alt="Photo of Shall64058" border="0" height="48" src="https://cdn.beeradvocate.com/data/avatars/s/1270/1270501.jpg?1560645799" width="48"/>,
 <img alt="Photo of Thisdrinkinglife" border="0" height="48" src="https://cdn.beeradvocate.com/data/avatars/s/903/903997.jpg?1417849501" width="48"/>,
 <img alt="Photo of DrBeergood" border="0" height="48" src="https://cdn.beeradvocate.com/data/avatars/s/1269/1269231.jpg?1561179290" width="48"/>,
 <img alt="Photo of Gurney_Halle

In [422]:
final = get_beer_info(final_list_links[0:10000])

https://www.beeradvocate.com/beer/profile/35/101/
https://www.beeradvocate.com/beer/profile/143/1352/
https://www.beeradvocate.com/beer/profile/35/14309/
https://www.beeradvocate.com/beer/profile/29/1252/
https://www.beeradvocate.com/beer/profile/35/68000/
https://www.beeradvocate.com/beer/profile/590/2565/
https://www.beeradvocate.com/beer/profile/35/74579/
https://www.beeradvocate.com/beer/profile/125/2272/
https://www.beeradvocate.com/beer/profile/337/1070/
https://www.beeradvocate.com/beer/profile/2137/3835/
https://www.beeradvocate.com/beer/profile/302/41422/
https://www.beeradvocate.com/beer/profile/66/1873/
https://www.beeradvocate.com/beer/profile/184/20299/
https://www.beeradvocate.com/beer/profile/428/14899/
https://www.beeradvocate.com/beer/profile/57/7714/
https://www.beeradvocate.com/beer/profile/417/47795/
https://www.beeradvocate.com/beer/profile/35/225611/
https://www.beeradvocate.com/beer/profile/107/7904/
https://www.beeradvocate.com/beer/profile/99/75739/
https://www

https://www.beeradvocate.com/beer/profile/5510/28676/
https://www.beeradvocate.com/beer/profile/544/45081/
https://www.beeradvocate.com/beer/profile/9374/70958/
https://www.beeradvocate.com/beer/profile/473/74305/
https://www.beeradvocate.com/beer/profile/15280/259728/
https://www.beeradvocate.com/beer/profile/20601/62870/
https://www.beeradvocate.com/beer/profile/5928/36013/
https://www.beeradvocate.com/beer/profile/3185/70851/
https://www.beeradvocate.com/beer/profile/36875/142038/
https://www.beeradvocate.com/beer/profile/546/5852/
https://www.beeradvocate.com/beer/profile/42049/401182/
https://www.beeradvocate.com/beer/profile/26093/84030/
https://www.beeradvocate.com/beer/profile/28510/314181/
https://www.beeradvocate.com/beer/profile/28086/78419/
https://www.beeradvocate.com/beer/profile/4667/13679/
https://www.beeradvocate.com/beer/profile/28018/113777/
https://www.beeradvocate.com/beer/profile/7295/85653/
https://www.beeradvocate.com/beer/profile/15368/57542/
https://www.beerad

https://www.beeradvocate.com/beer/profile/27835/261941/
https://www.beeradvocate.com/beer/profile/6663/46145/
https://www.beeradvocate.com/beer/profile/18680/67404/
https://www.beeradvocate.com/beer/profile/30626/322636/
https://www.beeradvocate.com/beer/profile/37582/170061/
https://www.beeradvocate.com/beer/profile/12511/110669/
https://www.beeradvocate.com/beer/profile/18086/49233/
https://www.beeradvocate.com/beer/profile/1619/25433/
https://www.beeradvocate.com/beer/profile/2641/21083/
https://www.beeradvocate.com/beer/profile/529/20319/
https://www.beeradvocate.com/beer/profile/555/264802/
https://www.beeradvocate.com/beer/profile/40554/200803/
https://www.beeradvocate.com/beer/profile/23473/74091/
https://www.beeradvocate.com/beer/profile/33873/373617/
https://www.beeradvocate.com/beer/profile/32560/247670/
https://www.beeradvocate.com/beer/profile/23058/77437/
https://www.beeradvocate.com/beer/profile/11175/283519/
https://www.beeradvocate.com/beer/profile/952/125571/
https://w

https://www.beeradvocate.com/beer/profile/6801/151574/
https://www.beeradvocate.com/beer/profile/2309/207134/
https://www.beeradvocate.com/beer/profile/5835/76319/
https://www.beeradvocate.com/beer/profile/30416/143391/
https://www.beeradvocate.com/beer/profile/31066/154655/
https://www.beeradvocate.com/beer/profile/27084/87584/
https://www.beeradvocate.com/beer/profile/16866/250400/
https://www.beeradvocate.com/beer/profile/38528/191008/
https://www.beeradvocate.com/beer/profile/1355/18465/
https://www.beeradvocate.com/beer/profile/6331/257570/
https://www.beeradvocate.com/beer/profile/34386/261156/
https://www.beeradvocate.com/beer/profile/57043/434214/
https://www.beeradvocate.com/beer/profile/4216/264999/
https://www.beeradvocate.com/beer/profile/21651/85545/
https://www.beeradvocate.com/beer/profile/6578/56617/
https://www.beeradvocate.com/beer/profile/43862/334892/
https://www.beeradvocate.com/beer/profile/19580/168236/
https://www.beeradvocate.com/beer/profile/24600/115244/
http

https://www.beeradvocate.com/beer/profile/40319/233983/
https://www.beeradvocate.com/beer/profile/44290/215808/
https://www.beeradvocate.com/beer/profile/49210/443136/
https://www.beeradvocate.com/beer/profile/44384/441089/
https://www.beeradvocate.com/beer/profile/10379/32515/
https://www.beeradvocate.com/beer/profile/16254/65540/
https://www.beeradvocate.com/beer/profile/41994/445444/
https://www.beeradvocate.com/beer/profile/705/267269/
https://www.beeradvocate.com/beer/profile/1659/178181/
https://www.beeradvocate.com/beer/profile/30534/255493/
https://www.beeradvocate.com/beer/profile/23623/61701/
https://www.beeradvocate.com/beer/profile/41807/290310/
https://www.beeradvocate.com/beer/profile/49682/393992/
https://www.beeradvocate.com/beer/profile/34731/315144/
https://www.beeradvocate.com/beer/profile/25329/68105/
https://www.beeradvocate.com/beer/profile/195/406025/
https://www.beeradvocate.com/beer/profile/482/265482/
https://www.beeradvocate.com/beer/profile/56625/427018/
htt

...

https://www.beeradvocate.com/beer/profile/32171/403502/
https://www.beeradvocate.com/beer/profile/18539/283182/
https://www.beeradvocate.com/beer/profile/13371/414766/
https://www.beeradvocate.com/beer/profile/26932/90158/
https://www.beeradvocate.com/beer/profile/39160/163118/
https://www.beeradvocate.com/beer/profile/31805/261934/
https://www.beeradvocate.com/beer/profile/47537/267823/
https://www.beeradvocate.com/beer/profile/39438/207663/
https://www.beeradvocate.com/beer/profile/73/407343/
https://www.beeradvocate.com/beer/profile/42756/215087/
https://www.beeradvocate.com/beer/profile/36243/230959/
https://www.beeradvocate.com/beer/profile/45990/234799/
https://www.beeradvocate.com/beer/profile/16271/244783/
https://www.beeradvocate.com/beer/profile/47052/447279/
https://www.beeradvocate.com/beer/profile/45814/258351/
https://www.beeradvocate.com/beer/profile/47043/255280/
https://www.beeradvocate.com/beer/profile/50467/407857/
https://www.beeradvocate.com/beer/profile/34076/4081

https://www.beeradvocate.com/beer/profile/36485/260165/
https://www.beeradvocate.com/beer/profile/44633/260421/
https://www.beeradvocate.com/beer/profile/50440/195397/
https://www.beeradvocate.com/beer/profile/33225/393542/
https://www.beeradvocate.com/beer/profile/34418/333638/
https://www.beeradvocate.com/beer/profile/11797/224326/
https://www.beeradvocate.com/beer/profile/3474/229958/
https://www.beeradvocate.com/beer/profile/40786/232518/
https://www.beeradvocate.com/beer/profile/2587/188230/
https://www.beeradvocate.com/beer/profile/52408/390470/
https://www.beeradvocate.com/beer/profile/55127/393543/
https://www.beeradvocate.com/beer/profile/47896/262983/
https://www.beeradvocate.com/beer/profile/38406/284743/
https://www.beeradvocate.com/beer/profile/34958/421447/
https://www.beeradvocate.com/beer/profile/50387/303687/
https://www.beeradvocate.com/beer/profile/43612/305991/
https://www.beeradvocate.com/beer/profile/27400/438855/
https://www.beeradvocate.com/beer/profile/51178/31

https://www.beeradvocate.com/beer/profile/7015/59225/
https://www.beeradvocate.com/beer/profile/41540/262234/
https://www.beeradvocate.com/beer/profile/47638/396890/
https://www.beeradvocate.com/beer/profile/18138/148570/
https://www.beeradvocate.com/beer/profile/25800/218458/
https://www.beeradvocate.com/beer/profile/50041/296282/
https://www.beeradvocate.com/beer/profile/46834/245338/
https://www.beeradvocate.com/beer/profile/39446/310874/
https://www.beeradvocate.com/beer/profile/27778/251482/
https://www.beeradvocate.com/beer/profile/32161/383066/
https://www.beeradvocate.com/beer/profile/39398/343131/
https://www.beeradvocate.com/beer/profile/29952/349787/
https://www.beeradvocate.com/beer/profile/2426/164187/
https://www.beeradvocate.com/beer/profile/43862/302171/
https://www.beeradvocate.com/beer/profile/54233/443227/
https://www.beeradvocate.com/beer/profile/140/450139/
https://www.beeradvocate.com/beer/profile/932/320091/
https://www.beeradvocate.com/beer/profile/38000/206940/

https://www.beeradvocate.com/beer/profile/50440/448116/
https://www.beeradvocate.com/beer/profile/18591/318068/
https://www.beeradvocate.com/beer/profile/53219/385140/
https://www.beeradvocate.com/beer/profile/40048/324980/
https://www.beeradvocate.com/beer/profile/51881/327284/
https://www.beeradvocate.com/beer/profile/46121/402293/
https://www.beeradvocate.com/beer/profile/55671/406133/
https://www.beeradvocate.com/beer/profile/44111/426101/
https://www.beeradvocate.com/beer/profile/43393/428149/
https://www.beeradvocate.com/beer/profile/47769/431477/
https://www.beeradvocate.com/beer/profile/37910/301173/
https://www.beeradvocate.com/beer/profile/35986/321653/
https://www.beeradvocate.com/beer/profile/11592/125301/
https://www.beeradvocate.com/beer/profile/34453/140662/
https://www.beeradvocate.com/beer/profile/44718/405366/
https://www.beeradvocate.com/beer/profile/35032/281974/
https://www.beeradvocate.com/beer/profile/4614/367990/
https://www.beeradvocate.com/beer/profile/3474/44

KeyboardInterrupt: 

In [423]:
df = pd.read_json('output.json')

In [425]:
df.shape

(7619, 13)

In [426]:
df.tail()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
7614,None,Rotating,4.00,NaN,Bellefonte Brewing Co.,https://cdn.beeradvocate.com/im/beers/101.jpg,Delaware,kush brown ale,1,0,[4/5 rDev 0%look: 4 | smell: 4 | taste: 4 | f...,American Brown Ale,https://www.beeradvocate.com/beer/profile/4569...
7615,5.9%,Rotating,3.85,NaN,Big Ugly Brewing Company,https://cdn.beeradvocate.com/im/beers/101.jpg,Virginia,Cooper's Ditch,1,1,[3.85/5 rDev 0%look: 4 | smell: 4 | taste: 3....,American Brown Ale,https://www.beeradvocate.com/beer/profile/3876...
7616,8%,Winter,4.00,NaN,Vertigo Brewing,https://cdn.beeradvocate.com/im/beers/101.jpg,Oregon,"Left, Nut Brown",1,0,"[4/5 rDev 0%Greels, Feb 21, 2014]",American Brown Ale,https://www.beeradvocate.com/beer/profile/1881...
7617,6.2%,Rotating,3.52,NaN,Castle Church Brewing Community,https://cdn.beeradvocate.com/im/beers/101.jpg,Florida,Means of Grace,1,0,[3.52/5 rDev 0%look: 3.75 | smell: 3.5 | tast...,American Brown Ale,https://www.beeradvocate.com/beer/profile/5487...
7618,None,Year-round,3.50,NaN,Erie Ale Works,https://cdn.beeradvocate.com/im/beers/101.jpg,Pennsylvania,Knob Twiddler,1,0,[3.5/5 rDev 0%look: 3.5 | smell: 3.5 | taste:...,American Brown Ale,https://www.beeradvocate.com/beer/profile/3934...


In [436]:
df.to_csv('scrape1.csv')

In [451]:
df0 = pd.read_csv('scrape1.csv', index_col=0)

In [452]:
df0.tail()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
7614,NaN,Rotating,4.00,NaN,Bellefonte Brewing Co.,https://cdn.beeradvocate.com/im/beers/101.jpg,Delaware,kush brown ale,1,0,['4/5\xa0\xa0rDev 0%look: 4 | smell: 4 | taste...,American Brown Ale,https://www.beeradvocate.com/beer/profile/4569...
7615,5.9%,Rotating,3.85,NaN,Big Ugly Brewing Company,https://cdn.beeradvocate.com/im/beers/101.jpg,Virginia,Cooper's Ditch,1,1,['3.85/5\xa0\xa0rDev 0%look: 4 | smell: 4 | ta...,American Brown Ale,https://www.beeradvocate.com/beer/profile/3876...
7616,8%,Winter,4.00,NaN,Vertigo Brewing,https://cdn.beeradvocate.com/im/beers/101.jpg,Oregon,"Left, Nut Brown",1,0,"['4/5\xa0\xa0rDev 0%Greels, Feb 21, 2014']",American Brown Ale,https://www.beeradvocate.com/beer/profile/1881...
7617,6.2%,Rotating,3.52,NaN,Castle Church Brewing Community,https://cdn.beeradvocate.com/im/beers/101.jpg,Florida,Means of Grace,1,0,['3.52/5\xa0\xa0rDev 0%look: 3.75 | smell: 3.5...,American Brown Ale,https://www.beeradvocate.com/beer/profile/5487...
7618,NaN,Year-round,3.50,NaN,Erie Ale Works,https://cdn.beeradvocate.com/im/beers/101.jpg,Pennsylvania,Knob Twiddler,1,0,['3.5/5\xa0\xa0rDev 0%look: 3.5 | smell: 3.5 |...,American Brown Ale,https://www.beeradvocate.com/beer/profile/3934...


In [432]:
df.loc[7618]['url']

'https://www.beeradvocate.com/beer/profile/39340/251513/'

In [435]:
final_list_links[7618]

'/beer/profile/39340/251513/'

In [439]:
final1 = get_beer_info(final_list_links[7618:15000])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [441]:
df = pd.read_json('output1.json')
df.to_csv('scrape2.csv')

In [442]:
df.tail(5)

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
7377,6.4%,Year-round,3.34,NaN,The Common Beer Company,https://cdn.beeradvocate.com/im/placeholder-be...,Ohio,Naughty Dark,1,0,[3.34/5 rDev 0%look: 3 | smell: 3.25 | taste:...,American Black Ale,https://www.beeradvocate.com/beer/profile/5420...
7378,5.5%,Rotating,3.40,NaN,4th Tap Brewing Co-op,https://cdn.beeradvocate.com/im/placeholder-be...,Texas,Black Lodge,1,0,[3.4/5 rDev 0%look: 3.5 | smell: 3.5 | taste:...,American Black Ale,https://www.beeradvocate.com/beer/profile/4310...
7379,6.6%,Rotating,3.92,NaN,Werk Force Brewing Co,https://cdn.beeradvocate.com/im/placeholder-be...,Illinois,Sex Panther,1,0,[3.92/5 rDev 0%look: 4 | smell: 3.75 | taste:...,American Black Ale,https://www.beeradvocate.com/beer/profile/3600...
7380,6.2%,One-off,4.14,NaN,Tattered Flag Brewery & Still Works,https://cdn.beeradvocate.com/im/beers/428849.jpg,Pennsylvania,The Moon Is A Dead World,1,1,[4.14/5 rDev 0%look: 4 | smell: 4.25 | taste:...,American Black Ale,https://www.beeradvocate.com/beer/profile/4653...
7381,8.3%,Rotating,3.28,NaN,Tin Lizard Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,Pennsylvania,Howl for Prey,1,0,[3.28/5 rDev 0%look: 3.75 | smell: 3.25 | tas...,American Black Ale,https://www.beeradvocate.com/beer/profile/4804...


In [443]:
final2 = get_beer_info(final_list_links[15000:35000]) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


ChunkedEncodingError: ('Connection broken: OSError("(54, \'ECONNRESET\')")', OSError("(54, 'ECONNRESET')"))

In [444]:
df2 = pd.read_json('output1.json')
df2.to_csv('scrape3.csv')

In [448]:
df2.tail(20)

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
7674,5.6%,Year-round,3.30,NaN,Font Salem,https://cdn.beeradvocate.com/im/beers/177490.jpg,Spain,Argus Negra,2,2,[4.59/5 rDev +39.1%look: 5 | smell: 4 | taste...,European Dark Lager,https://www.beeradvocate.com/beer/profile/1509...
7675,None,Year-round,2.73,NaN,Hoa Vien Brauhaus Restaurant,https://cdn.beeradvocate.com/im/placeholder-be...,Viet Nam,Bia Đen,2,2,[2.61/5 rDev -4.4%look: 4 | smell: 2.25 | tas...,European Dark Lager,https://www.beeradvocate.com/beer/profile/6432...
7676,3.8%,Year-round,3.70,NaN,Pivovar Rakovnik,https://cdn.beeradvocate.com/im/beers/345175.jpg,Czech Republic,Bakalar tmava Desitka,2,0,[3.77/5 rDev +1.9%look: 4 | smell: 3.75 | tas...,European Dark Lager,https://www.beeradvocate.com/beer/profile/1659...
7677,5.8%,Year-round,4.16,NaN,De Tacchi,https://cdn.beeradvocate.com/im/placeholder-be...,Italy,Maranello - Special Amber Beer,2,2,[5/5 rDev +20.2%look: 5 | smell: 5 | taste: 5...,European Dark Lager,https://www.beeradvocate.com/beer/profile/2651...
7678,5%,Year-round,3.41,NaN,Pinchgut Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,Australia,Black Pilsner,2,2,[3.48/5 rDev +2.1%look: 4 | smell: 3.5 | tast...,European Dark Lager,https://www.beeradvocate.com/beer/profile/2733...
7679,4.5%,Year-round,3.93,NaN,Obec Brewing,https://cdn.beeradvocate.com/im/placeholder-be...,Washington,Czech Dark,2,1,[3.84/5 rDev -2.3%look: 4 | smell: 3.75 | tas...,European Dark Lager,https://www.beeradvocate.com/beer/profile/5029...
7680,None,Year-round,3.38,NaN,Hercegovačka Pivovara (Heppro),https://cdn.beeradvocate.com/im/placeholder-be...,Bosnia & Herzegovina,Kosaca,2,0,[3.5/5 rDev +3.6%look: 3.5 | smell: 3.5 | tas...,European Dark Lager,https://www.beeradvocate.com/beer/profile/3409...
7681,4.8%,Year-round,3.00,NaN,Cereuro - Cervejeira Europeia (Tagus),https://cdn.beeradvocate.com/im/beers/44639.jpg,Portugal,Tagus Preta,2,2,[3.35/5 rDev +11.7%look: 3 | smell: 3 | taste...,European Dark Lager,https://www.beeradvocate.com/beer/profile/1320...
7682,5%,Year-round,2.85,NaN,Privatbrauerei Fritz Egger GmbH & Co,https://cdn.beeradvocate.com/im/beers/203618.jpg,Austria,Kenner Dark Beer,2,0,[2.5/5 rDev -12.3%look: 2.5 | smell: 2.5 | ta...,European Dark Lager,https://www.beeradvocate.com/beer/profile/4460...
7683,7.3%,Rotating,3.97,NaN,Bellefonte Brewing Co.,https://cdn.beeradvocate.com/im/placeholder-be...,Delaware,Nova Luna,2,0,[4/5 rDev +0.8%look: 4 | smell: 4 | taste: 4 ...,European Dark Lager,https://www.beeradvocate.com/beer/profile/4569...


In [453]:
len(df0)

7619

In [455]:
len(df)

7382

In [456]:
len(df2)

7694

In [457]:
7619+7382+7694

22695